## Dissertation Demo - Random Search Algorithm
* Data: User = User2
* Meal Vector: MV1

* Calorie Constraint
    * Calorie Constraint: 1800
    * Calorie Target Range: +/- 20%

* MicroNutrient Targets/Lower Bound: 
    * vitamin a IU: 2333
    * vitamin b6:   1.3
    * vitamin c:    75
    * niacin:       14
    * folate:	    400
    * calcium:      1000
    * iron:         18
    * magnesium:    320
    * potassium:    2600	
    * sodium:	    1500
    * thiamin:      1.1
    * fiber (g):    25    
    
* Nutrient Targets/Upper Bound: 
    * cholesterol:  300
    * saturate fat: <= 10% calories
    * sugar:        40
    * sodium:	    2300
   
* Rating Adjustment

* Add meals to vector until calorie target reached
* Algorithm: Genetic Algorithm:
    * Population: 100
    * Generations: 10
    * Parent Selection: Tournament, k=5
    * Crossover: Uniform Crossover, p_cross=0.25
    * Mutation: p_mut = 0.2
    * Survivor Selection: Best of parent, child, mutant

In [ ]:
######################################################################
# Library Import, Data Import, Meal List Creation, Variable Creation
######################################################################
import random 
import copy
import numpy as np
import pandas as pd
import time
import warnings
warnings.filterwarnings("ignore")
import requests
import matplotlib.pylab as plt
%matplotlib inline
plt.rcParams.update({'figure.figsize':(8,5), 'figure.dpi':100})
import matplotlib.image as mpimg
from matplotlib import rcParams
from PIL import Image

df = pd.read_excel("UserNo2.xlsx")

# Define the data frame
selected_features = ['headline', 'Breakfast', 'Lunch', 'Dinner', 'Side', 'Dessert', 'Snack',
                     'vitamin a iu', 'niacin', 'vitamin b6', 'vitamin c', 'folate', 'calcium', 'iron',
                     'magnesium', 'potassium', 'sodium', 'thiamin', 'cms_id', 'calories', 'rating',
                     'carbohydrateContent', 'cholesterolContent', 'fatContent', 'fiberContent', 'proteinContent',
                     'saturatedFatContent', 'sodiumContent', 'sugarContent', 'ingredients', 'primary_image',
                     'servings', 'prep_time', 'cook_time', 'total_time']

# Set up meal lists
user1List = df[selected_features]
breakfast = user1List.loc[user1List['Breakfast'] == 1]['cms_id'].tolist()
lunch = user1List.loc[user1List['Lunch'] == 1]['cms_id'].tolist()
dinner = user1List.loc[user1List['Dinner'] == 1]['cms_id'].tolist()
side = user1List.loc[user1List['Side'] == 1]['cms_id'].tolist()
dessert = user1List.loc[user1List['Dessert'] == 1]['cms_id'].tolist()
snack = user1List.loc[user1List['Snack'] == 1]['cms_id'].tolist()

# Nutrient Constraints
target_cal = 1800
ul_sod = 2300
ul_chol = 300
ul_satfat = 0.1 * target_cal/9
ul_sugar = 40

# Set up algorithm variables
food = []
rating = []
cal = []
fat = []
pro = []
carb = []
fiber = []
sugar = []
chol = []
satfat = []
netcarb = []
vita = []
niacin = []
vitb6 = []
vitc = []
folate = []
calcium = []
iron = []
magn = []
pot = []
sod = []
thiamin = []
total_cal = []
total_fat = []
total_pro = []
total_carb = []
total_fiber = []
total_sugar = []
total_chol = []
total_satfat = []
total_netcarb = []
mean_rating = []
total_vita = []
total_niacin = []
total_vitb6 = []
total_vitc = []
total_folate = []
total_calcium = []
total_iron = []
total_magn = []
total_pot = []
total_sod = []
total_thiamin = []   
nnr = []
x_list = []
food_list = []
leader = []
rs_pop = {}

In [ ]:
#################################################################
# Nutrition Fitness Function 
#################################################################
def fitness(v):
    rating = []
    cal = []
    fat = []
    pro = []
    carb = []
    fiber = []
    sugar = []
    chol = []
    satfat = []
    netcarb = []
    vita = []
    niacin = []
    vitb6 = []
    vitc = []
    folate = []
    calcium = []
    iron = []
    magn = []
    pot = []
    sod = []
    thiamin = []
    # Define nutrient parameters for food list
    for i in range(len(v)):
        rating.append(user1List.loc[user1List['cms_id'] == v[i]]['rating'])
        cal.append(user1List.loc[user1List['cms_id'] == v[i]]['calories'])
        fat.append(user1List.loc[user1List['cms_id'] == v[i]]['fatContent'])
        pro.append(user1List.loc[user1List['cms_id'] == v[i]]['proteinContent'])
        carb.append(user1List.loc[user1List['cms_id'] == v[i]]['carbohydrateContent'])
        fiber.append(user1List.loc[user1List['cms_id'] == v[i]]['fiberContent'])
        sugar.append(user1List.loc[user1List['cms_id'] == v[i]]['sugarContent'])
        chol.append(user1List.loc[user1List['cms_id'] == v[i]]['cholesterolContent'])
        satfat.append(user1List.loc[user1List['cms_id'] == v[i]]['saturatedFatContent'])
        netcarb.append(netcarb)
        vita.append(user1List.loc[user1List['cms_id'] == v[i]]['vitamin a iu'])
        niacin.append(user1List.loc[user1List['cms_id'] == v[i]]['niacin'])
        vitb6.append(user1List.loc[user1List['cms_id'] == v[i]]['vitamin b6'])
        vitc.append(user1List.loc[user1List['cms_id'] == v[i]]['vitamin c'])
        folate.append(user1List.loc[user1List['cms_id'] == v[i]]['folate'])
        calcium.append(user1List.loc[user1List['cms_id'] == v[i]]['calcium'])        
        iron.append(user1List.loc[user1List['cms_id'] == v[i]]['iron'])
        magn.append(user1List.loc[user1List['cms_id'] == v[i]]['magnesium'])
        pot.append(user1List.loc[user1List['cms_id'] == v[i]]['potassium'])
        sod.append(user1List.loc[user1List['cms_id'] == v[i]]['sodium'])
        thiamin.append(user1List.loc[user1List['cms_id'] == v[i]]['thiamin'])       

    #Calculate total and percent values for food list
    total_cal = round(np.sum(cal))
    total_fat = round(np.sum(fat), 1)
    total_pro = round(np.sum(pro), 1)
    total_carb = round(np.sum(carb), 1)
    total_fiber = round(np.sum(fiber), 1)
    total_sugar = round(np.sum(sugar), 1)
    total_chol = round(np.sum(chol), 1)
    total_satfat = round(np.sum(satfat), 1)
    total_netcarb = round((total_carb - total_fiber), 1)
    mean_rating = round(np.mean(rating), 2)
    total_vita = round(np.sum(vita), 1)
    total_niacin = round(np.sum(niacin), 2)
    total_vitb6 = round(np.sum(vitb6), 2)
    total_vitc = round(np.sum(vitc), 2)
    total_folate = round(np.sum(folate), 2)
    total_calcium = round(np.sum(calcium), 2)
    total_iron = round(np.sum(iron), 2)
    total_magn = round(np.sum(magn), 2)
    total_pot = round(np.sum(pot), 2)
    total_sod = round(np.sum(sod), 2)
    total_thiamin = round(np.sum(thiamin), 2)      
    p_cal = round(total_cal / target_cal, 2)
    p_vita = round(total_vita / 2333, 2)
    p_niacin = round(total_niacin / 14, 2)
    p_vitb6 = round(total_vitb6 / 1.3, 2)
    p_vitc = round(total_vitc / 75, 2)
    p_folate = round(total_folate / 400, 2)
    p_calcium = round(total_calcium / 1000, 2)
    p_iron = round(total_iron / 18, 2)
    p_magn = round(total_magn / 320, 2)
    p_pot = round(total_pot / 2600, 2)
    p_sod = round(total_sod / 1500, 2)
    p_thiamin = round(total_thiamin / 1.1, 2)
    n_fiber = round(total_fiber / 25, 2)

    #Calculate NNR with each value truncated at 100%
    nnr = round(((min(1,  p_vita) + min(1, p_niacin) + min(1, p_vitb6) + min(1, p_vitc) + 
                  min(1, p_folate) + min(1, n_fiber) + min(1, p_calcium) + min(1, p_iron) + 
                  min(1, p_magn) + min(1, p_pot) + min(1, p_sod) + min(1, p_thiamin))/12), 2)

    #Calculate Upper Bound Penalty###############################################################
    # Sodium penalty
    if total_sod > 2300:
        sod_pen = round(np.absolute(((np.absolute(total_sod - 2300)**2./3.))/3526465), 5)
    else:
        sod_pen = 0
    # Cholesterol penalty
    if total_chol > 300:
        chol_pen = round(np.absolute(((np.absolute(total_chol - 300)**2./3.))/59952), 5)
    else:
        chol_pen = 0
    # Saturated Fat penalty
    if total_satfat > 0.1 * target_cal/9:
        satfat_pen = round(np.absolute(((np.absolute(total_satfat - 0.1 * target_cal/9)**2./3.))/533), 5)
    else:
        satfat_pen = 0
    # Sugar penalty
    if total_sugar > 40:
        sugar_pen = round(np.absolute(((np.absolute(total_sugar - 40)**2./3.))/1066), 5)
    else:
        sugar_pen = 0       
        
    # Sum of penalty functions
    ubp = sod_pen + chol_pen + satfat_pen + sugar_pen
    
    #Calorie Penalty Function (allows up to 110% of target then applies a penalty)################
    if total_cal < 0.8*target_cal or total_cal > 1.2*target_cal:
        cal_pen = 0
    else:
        cal_pen = 1  

    #Rating Penalty Function  ################
    rating_pen = -0.05 * (mean_rating * mean_rating) + (0.55 * mean_rating) - 0.5

    score = round((nnr - ubp) * (cal_pen) * (rating_pen), 4)
    return score

In [ ]:
#################################################################
# RS Flow for Population Size and Generations (Fitness Driven)
# Run several generations and output best solution
#################################################################
def rs_alg(iterations, pop_size):
    best_key = float(0)
    print()
    print("{:<6} {:<7} {:<9} {:<68} {:<7}".format("Iter", "Steps", "Fitness", "Leader Meal Vector", "Ex Time"))
    print("{:<6} {:<7} {:<9} {:<68} {:<7}".format("----", "-----", "-------", "----------------------------------------------------------------", "-------"))   
    for m in range(1, iterations + 1):
        x_list = []
        food_list = []
        lead_key = []
        leader = []
        startTime = time.time() #Start iteration timer
        for j in range(pop_size):
            v = [random.choice(breakfast), random.choice(lunch), random.choice(dinner), random.choice(side), 
                 random.choice(side), random.choice(snack), random.choice(snack), random.choice(dessert)]
            food = []
            cal = []
            for i in range(len(v)):
                cal.append(user1List.loc[user1List['cms_id'] == v[i]]['calories'])
                total_cal = np.sum(cal)
                if total_cal < target_cal:
                    food.append(v[i])
                else:
                    food.append(999999)
            x = fitness(food)
            x_list.append(x)
            food_list.append(food)
            total_cal==0  
        rs_pop = dict(zip(x_list, food_list))
        lead_key = np.max(list(rs_pop.keys())) # Selects top individual fitness from population
        leader = rs_pop[lead_key] # Top individual from population
        exTime = round((time.time() - startTime),2)
        print("{:<6} {:<7} {:<9} {:<68} {:<7}".format(m, pop_size, round(lead_key,4), str(leader), exTime))
        if lead_key > best_key:
            best_key = lead_key
            best_leader = leader
        # Early Stop routine to end algorithm early if fitness requirement met
        if best_key <= 1.0:
            continue            
        else:
            print('early stop triggered')
            break
    return best_leader, best_key

In [ ]:
#################################################################
# Vector Output Function 
#################################################################
# Reformatted decoder with macronutrients and penalties on top and vitamins and minerals on bottom
def decode(v):    
    # Define Variables
    food = []
    rating = []
    cal = []
    fat = []
    pro = []
    carb = []
    fiber = []
    sugar = []
    chol = []
    satfat = []
    netcarb = []
    vita = []
    niacin = []
    vitb6 = []
    vitc = []
    folate = []
    calcium = []
    iron = []
    magn = []
    pot = []
    sod = []
    thiamin = []
    total_cal = []
    total_fat = []
    total_pro = []
    total_carb = []
    total_fiber = []
    total_sugar = []
    total_chol = []
    total_satfat = []
    total_netcarb = []
    mean_rating = []
    total_vita = []
    total_niacin = []
    total_vitb6 = []
    total_vitc = []
    total_folate = []
    total_calcium = []
    total_iron = []
    total_magn = []
    total_pot = []
    total_sod = []
    total_thiamin = []   
    nnr = []

    # Lookup for each recipe
    for i in range(len(v)):
        rating.append(user1List.loc[user1List['cms_id'] == v[i]]['rating'])
        cal.append(user1List.loc[user1List['cms_id'] == v[i]]['calories'])
        fat.append(user1List.loc[user1List['cms_id'] == v[i]]['fatContent'])
        pro.append(user1List.loc[user1List['cms_id'] == v[i]]['proteinContent'])
        carb.append(user1List.loc[user1List['cms_id'] == v[i]]['carbohydrateContent'])
        fiber.append(user1List.loc[user1List['cms_id'] == v[i]]['fiberContent'])
        sugar.append(user1List.loc[user1List['cms_id'] == v[i]]['sugarContent'])
        chol.append(user1List.loc[user1List['cms_id'] == v[i]]['cholesterolContent'])
        satfat.append(user1List.loc[user1List['cms_id'] == v[i]]['saturatedFatContent'])
        netcarb.append(netcarb)
        vita.append(user1List.loc[user1List['cms_id'] == v[i]]['vitamin a iu'])
        niacin.append(user1List.loc[user1List['cms_id'] == v[i]]['niacin'])
        vitb6.append(user1List.loc[user1List['cms_id'] == v[i]]['vitamin b6'])
        vitc.append(user1List.loc[user1List['cms_id'] == v[i]]['vitamin c'])
        folate.append(user1List.loc[user1List['cms_id'] == v[i]]['folate'])
        calcium.append(user1List.loc[user1List['cms_id'] == v[i]]['calcium'])        
        iron.append(user1List.loc[user1List['cms_id'] == v[i]]['iron'])
        magn.append(user1List.loc[user1List['cms_id'] == v[i]]['magnesium'])
        pot.append(user1List.loc[user1List['cms_id'] == v[i]]['potassium'])
        sod.append(user1List.loc[user1List['cms_id'] == v[i]]['sodium'])
        thiamin.append(user1List.loc[user1List['cms_id'] == v[i]]['thiamin'])       

    #Calculate total and percent values for food list
    total_cal = round(np.sum(cal), 1)
    total_fat = round(np.sum(fat), 1)
    total_pro = round(np.sum(pro), 1)
    total_carb = round(np.sum(carb), 1)
    total_fiber = round(np.sum(fiber), 1)
    total_sugar = round(np.sum(sugar), 1)
    total_chol = round(np.sum(chol), 1)
    total_satfat = round(np.sum(satfat), 1)
    total_netcarb = round((total_carb - total_fiber), 1)
    mean_rating = round(np.mean(rating), 2)
    total_vita = round(np.sum(vita), 1)
    total_niacin = round(np.sum(niacin), 2)
    total_vitb6 = round(np.sum(vitb6), 2)
    total_vitc = round(np.sum(vitc), 2)
    total_folate = round(np.sum(folate), 2)
    total_calcium = round(np.sum(calcium), 2)
    total_iron = round(np.sum(iron), 2)
    total_magn = round(np.sum(magn), 2)
    total_pot = round(np.sum(pot), 2)
    total_sod = round(np.sum(sod), 2)
    total_thiamin = round(np.sum(thiamin), 2)      
    p_cal = round(total_cal / target_cal, 2)
    p_vita = round(total_vita / 2333, 2)
    p_niacin = round(total_niacin / 14, 2)
    p_vitb6 = round(total_vitb6 / 1.3, 2)
    p_vitc = round(total_vitc / 75, 2)
    p_folate = round(total_folate / 400, 2)
    p_calcium = round(total_calcium / 1000, 2)
    p_iron = round(total_iron / 18, 2)
    p_magn = round(total_magn / 320, 2)
    p_pot = round(total_pot / 2600, 2)
    p_sod = round(total_sod / 1500, 2)
    p_thiamin = round(total_thiamin / 1.1, 2)
    n_fiber = round(total_fiber / 25, 2)
    score = best_key
   
    # Print Fitness Score and Mean Rating
    print(f'Fitness Score: {round(score,4)}')    
    print(f'Menu Rating: {mean_rating} out of 5')   
    
    # Food Menu Lookup Output Header - Macronutrients
    print("{:<31} {:<8} {:<7} {:<7} {:<7} {:<7} {:<7} {:<7} {:<7} {:<8} {:<8} {:<7}".format('Name','Rating','Cal','Fat','Pro','Carb','NetCarb','Fiber','Chol','Sat Fat','Sodium','Sugar'))
    print("{:<31} {:<8} {:<7} {:<7} {:<7} {:<7} {:<7} {:<7} {:<7} {:<8} {:<8} {:<7}".format('----','------','---','---','---','----','-------','-----','----','-------','------','-----'))        
    for i in v:
        ind = user1List.loc[user1List['cms_id'] == i].index[0]
        name = user1List['headline'][ind][0:29] #truncates to first 29 characters of name string
        rating = user1List['rating'][ind]
        cal = user1List['calories'][ind]
        fat = user1List['fatContent'][ind]
        satfat = user1List['saturatedFatContent'][ind]
        chol = user1List['cholesterolContent'][ind]
        pro = user1List['proteinContent'][ind]
        carb = user1List['carbohydrateContent'][ind]
        fiber = user1List['fiberContent'][ind]
        netcarb = round((carb - fiber), 1)
        sod = user1List['sodium'][ind]
        sugar = user1List['sugarContent'][ind]
        vita = user1List['vitamin a iu'][ind]
        niacin = user1List['niacin'][ind]
        vitb6 = user1List['vitamin b6'][ind]
        vitc = user1List['vitamin c'][ind]
        folate = user1List['folate'][ind]
        print("{:<31} {:<8} {:<7} {:<7} {:<7} {:<7} {:<7} {:<7} {:<7} {:<8} {:<8} {:<7}".format(name, rating, round(cal,1), round(fat,1), round(pro,1), round(carb,1), round(netcarb,1), round(fiber,1), round(chol,1), round(satfat,1), round(sod,1), round(sugar,1)))
    print("-----------------------------------------------------------------------------------------------------------------------")
    p_fat = round((((total_fat * 9) / total_cal) *100), 1)
    p_pro = round((((total_pro * 4) / total_cal) *100), 1)
    p_carb = round((((total_carb * 4) / total_cal) *100), 1)
    p_fiber = round((((total_fiber * 4) / total_cal) *100), 1)
    p_netcarb = round((((total_netcarb * 4) / total_cal) *100), 1)
    p_sugar = round((((total_sugar * 4) / total_cal) *100), 1)
    utl_sod = round(((total_sod / ul_sod) *100), 1)
    utl_chol = round(((total_chol / ul_chol) *100), 1)
    utl_satfat = round(((total_satfat / ul_satfat) *100), 1)
    utl_sugar = round(((total_sugar / ul_sugar) *100), 1)
    print("{:<31} {:<8} {:<7} {:<7} {:<7} {:<7} {:<7} {:<7} {:<7} {:<8} {:<8} {:<7}".format("Total", mean_rating, total_cal, total_fat, total_pro, total_carb, total_netcarb, total_fiber, total_chol, total_satfat, total_sod, total_sugar))
    print("{:<31} {:<8} {:<7} {:<7} {:<7} {:<7} {:<7} {:<7} {:<7} {:<8} {:<8} {:<7}".format("% Calories", "","", p_fat, p_pro, p_carb, p_netcarb, p_fiber, "", "", "", p_sugar))
    print("{:<31} {:<8} {:<7} {:<7} {:<7} {:<7} {:<7} {:<7} {:<7} {:<8} {:<8} {:<7}".format("% UTL", "", round(p_cal*100,1), "", "", "", "", "", utl_chol, utl_satfat, utl_sod, utl_sugar))
    print()
    # Food Menu Lookup Output Header - Vitamins and Minerals
    print ("{:<31} {:<8} {:<7} {:<7} {:<7} {:<7} {:<7} {:<7} {:<7} {:<8} {:<8} {:<7}".format('Name','Vit A','Vit B6','Vit C','Niac','Fol','Calc','Iron','Magn','Pot','Sodium','Thia'))
    print ("{:<31} {:<8} {:<7} {:<7} {:<7} {:<7} {:<7} {:<7} {:<7} {:<8} {:<8} {:<7}".format('----','-----','------','-----','----','---','----','----','----','---','------','---'))
    for i in v:
        ind = user1List.loc[user1List['cms_id'] == i].index[0]
        name = user1List['headline'][ind][0:29] #truncates to first 29 characters of name string
        vita = user1List['vitamin a iu'][ind]
        niacin = user1List['niacin'][ind]
        vitb6 = user1List['vitamin b6'][ind]
        vitc = user1List['vitamin c'][ind]
        folate = user1List['folate'][ind]
        calcium = user1List['calcium'][ind]        
        iron = user1List['iron'][ind]
        magn = user1List['magnesium'][ind]
        pot = user1List['potassium'][ind]
        sod = user1List['sodium'][ind]
        thiamin = user1List['thiamin'][ind]   
        print("{:<31} {:<8} {:<7} {:<7} {:<7} {:<7} {:<7} {:<7} {:<7} {:<8} {:<8} {:<7}".format(name, round(vita,1), round(vitb6,1), round(vitc,1), round(niacin,1), round(folate,1), round(calcium,1), round(iron,1), round(magn,1), round(pot,1), round(sod,1), round(thiamin,1)))
    print("-----------------------------------------------------------------------------------------------------------------------")
    print("{:<31} {:<8} {:<7} {:<7} {:<7} {:<7} {:<7} {:<7} {:<7} {:<8} {:<8} {:<7}".format("Total", round(total_vita,1), round(total_vitb6,1), round(total_vitc,1), round(total_niacin,1), round(total_folate,1), round(total_calcium,1), round(total_iron,1), round(total_magn,1), round(total_pot,1), round(total_sod,1), round(total_thiamin,1)))
    print("{:<31} {:<8} {:<7} {:<7} {:<7} {:<7} {:<7} {:<7} {:<7} {:<8} {:<8} {:<7}".format("% RDA", round(p_vita*100,1), round(p_vitb6*100,1), round(p_vitc*100,1), round(p_niacin*100,1), round(p_folate*100,1), round(p_calcium*100,1), round(p_iron*100,1), round(p_magn*100,1), round(p_pot*100,1), round(p_sod*100,1), round(p_thiamin*100,1)))
    print('_______________________________________________________________________________________________________________________')
    #Calculate Flags###############################################################
    # Sodium penalty
    if total_sod > 2300:
        sod_flag = 1
    else:
        sod_flag = 0
    # Cholesterol penalty
    if total_chol > 300:
        chol_flag = 1
    else:
        chol_flag = 0
    # Saturated Fat penalty
    if total_satfat > 0.1 * target_cal/9:
        satfat_flag = 1
    else:
        satfat_flag = 0
    # Sugar penalty
    if total_sugar > 40:
        sugar_flag = 1
    else:
        sugar_flag = 0              
    # Fiber flag (flag only, no penalty here (included in nnr))
    if total_fiber < 25:
        fiber_flag = 1
    else:
        fiber_flag = 0

    print('Flags:')
    if chol_flag == 1:
        print(f'Cholesterol is {total_chol}mg and exceeds daily recommended upper limit of 300mg')
    if satfat_flag == 1:
        print(f'Total Saturated Fat is {total_satfat}g and exceeds daily recommended upper limit of {round(0.1*target_cal/9,1)}g')
    if sod_flag == 1:
        print(f'Total Sodium is {total_sod}mg and exceeds daily recommended upper limit of 2300mg')
    if sugar_flag == 1:
        print(f'Total Sugar is {total_sugar}mg and exceeds daily recommended upper limit of 40mg')
    if fiber_flag == 1:
        print(f'Total Fiber is {total_fiber}g and is less than the daily recommended requirement of 25g')        
    print('_______________________________________________________________________________________________________________________')
    print()
    print()

In [ ]:
#################################################################
# Display Menu
#################################################################
from textwrap import wrap

def display_recipe(cms):
    chars = "'[]"
    ind = user1List.loc[user1List['cms_id'] == cms].index[0]
    name = (user1List['headline'][ind])
    ingredients = (user1List['ingredients'][ind])
    servings = round((user1List['servings'][ind]))
    prep_time = (user1List['prep_time'][ind])
    cook_time = (user1List['cook_time'][ind])
    total_time = (user1List['total_time'][ind])
    ing = ingredients.translate(str.maketrans('', '', chars))
    print(name)
    image_url = (user1List['primary_image'][ind])
    response = requests.get(image_url)
    image = Image.open(requests.get(image_url, stream=True).raw)
    image.thumbnail((300, 300))
    display(image)
    print(f'Servings: {servings}')
    print(f'Prep Time: {prep_time}, Cook Time: {cook_time}, Total Time: {total_time}')
    print()
    print('Ingredients:')
    lines = wrap(ing, 60) #wraps output as a list of lines
    print('\n'.join(lines)) 
    return cms
    
def display_menu(list):
    meal_name = ['Breakfast', 'Lunch', 'Dinner', 'Side Dish', 'Side Dish', 'Snack', 'Snack', 'Dessert']
    for i in range(len(list)):
        if list[i] == 999999:
            continue
        else:
            print(meal_name[i])
            display_recipe(list[i])
            print()
            print('____________________________________________________________')

# Simulation for Use Case 2

In [ ]:
best, best_key = rs_alg(iterations = 1, pop_size = 5000)
print()
print(f'Best Solution: {best_key, best}')
print()
decode(best)
display_menu(best)

## References
[1] Search code adapted from MATPMD4 lecture and practical materials <br />
[2] Execution time code adapted from: https://datatofish.com/measure-time-to-run-python-script/ <br />
[3] Formatting code adapted from: https://www.educba.com/python-print-table/ <br />
[4] Random Library - https://docs.python.org/3/library/random.html <br />
[5] Copy Library - https://docs.python.org/3/library/copy.html <br />
[6] NumPy Library - https://numpy.org/doc/stable/reference/ <br />
[7] Pandas Library - https://pandas.pydata.org/ <br />
[8] Time Library - https://docs.python.org/3/library/time.html <br />
[9] Warnings Library - https://docs.python.org/3/library/warnings.html <br />
[10] Matplotlib Library - https://matplotlib.org/ <br />
[11] Requests Library - https://realpython.com/python-requests/ <br />
[12] Pillow Library - https://pypi.org/project/Pillow/ <br />